# 2.3 Subconsultas Avanzadas

**Resumen Ejecutivo**: Las subconsultas correlacionadas y `EXISTS/NOT EXISTS` permiten filtros contextuales complejos; aprende cuándo usarlas y cuándo reescribirlas con JOINs o CTEs para mejorar rendimiento y legibilidad.

---

## 🎯 Objetivos de Aprendizaje
- Aplicar subconsultas correlacionadas para comparaciones con promedios segmentados.
- Usar `EXISTS` para verificar existencia sin traer datos.
- Reescribir subconsultas con CTEs/JOINs cuando mejore claridad o rendimiento.

## 🧰 Prerrequisitos
- Haber completado `02_ctes_recursivas.ipynb`.
- Dominar JOINs y agregaciones (`GROUP BY`, `HAVING`).
- Conocer el impacto de índices en columnas filtradas.

---

## 📚 Conceptos Clave
- **Subconsulta correlacionada**: subconsulta que referencia tabla externa (se ejecuta por cada fila).
- **EXISTS**: verifica existencia de filas (eficiente con índices, retorna booleano).
- **NOT EXISTS**: filtro negativo (filas sin coincidencias).
- **Reescritura**: convertir correlaciones en JOINs/CTEs para evitar ejecuciones repetidas.

> ⚠️ Subconsultas correlacionadas profundas degradan rendimiento; falta de índices en columnas filtradas amplifica el problema.

In [ ]:
-- Subconsulta correlacionada: productos con margen total > promedio global
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE (SELECT SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario))
       FROM dbo.fact_ventas fv
       WHERE fv.producto_id = p.producto_id) > (
          SELECT AVG(sub.margen_prod)
          FROM (SELECT producto_id, SUM(cantidad*(precio_unitario-costo_unitario)) AS margen_prod
                FROM dbo.fact_ventas fv2
                JOIN dbo.dim_productos p2 ON fv2.producto_id = p2.producto_id
                GROUP BY producto_id) sub
      );

In [ ]:
-- EXISTS: productos con al menos una venta con descuento >30%
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE EXISTS (SELECT 1 FROM dbo.fact_ventas fv WHERE fv.producto_id = p.producto_id AND fv.descuento_pct > 30);

---

## 🧪 Ejercicios Guiados
- 🟢 Productos con ventas en todos los días registrados (comparar `COUNT(DISTINCT fecha)`).
- 🟠 Clientes cuyo total de unidades supera el promedio de su segmento (subconsulta correlacionada).
- 🔴 Reescribir ejercicio anterior con CTE y JOIN evitando subconsulta correlacionada.

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE (SELECT COUNT(DISTINCT fecha) FROM dbo.fact_ventas fv WHERE fv.producto_id = p.producto_id) = (SELECT COUNT(DISTINCT fecha) FROM dbo.fact_ventas);

In [ ]:
SELECT c.cliente_id, c.nombre, c.segmento
FROM dbo.dim_clientes c
WHERE (SELECT SUM(fv.cantidad) FROM dbo.fact_ventas fv WHERE fv.cliente_id = c.cliente_id) > (
    SELECT AVG(sub.total_unidades)
    FROM (SELECT cliente_id, SUM(cantidad) AS total_unidades
          FROM dbo.fact_ventas fv2
          GROUP BY cliente_id) sub
    JOIN dbo.dim_clientes c2 ON sub.cliente_id = c2.cliente_id
    WHERE c2.segmento = c.segmento
);

---

## ⚠️ Errores Comunes
- Subconsultas innecesarias cuando un JOIN cumple el mismo propósito y es más eficiente.
- Correlaciones profundas sin índices adecuados degradan rendimiento exponencialmente.
- Olvidar que `EXISTS` solo verifica existencia; si necesitas datos, usa JOIN.
- No validar planes de ejecución para identificar cuellos de botella en subconsultas.

---

## ✅ Conclusiones
- Subconsultas correlacionadas resuelven comparaciones contextuales pero pueden degradar rendimiento.
- `EXISTS/NOT EXISTS` es eficiente para verificaciones booleanas con índices adecuados.
- Reescribir con CTEs/JOINs mejora legibilidad y a menudo rendimiento; validar planes de ejecución.

---

## 🚀 Aplicación Práctica
- Identificar productos con ventas consistentes (todos los días).
- Segmentar clientes por comportamiento relativo a su cohorte.
- Detectar outliers (clientes/productos con métricas atípicas vs. su segmento).

---

## 💼 Perspectiva de Negocio
- Comparaciones segmentadas permiten identificar top performers dentro de cada categoría.
- Detección de anomalías contextuales (vs. promedio de segmento) previene alertas falsas.
- Refactorización proactiva de subconsultas costosas reduce costos de infraestructura y mejora UX en dashboards.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 2  
**Módulo**: 2.3 Subconsultas Avanzadas  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](02_ctes_recursivas.ipynb) | [Siguiente ➡️](04_modelado_dimensional_basico.ipynb)
---
